In [349]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from import_file import  *
load_libs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 最低级别段的计算
## 读取k线数据

In [324]:
instrument = Instrument()
code = 'il8'
df = instrument.bar(code, asset='future')
hl_df = df.loc[:, ['high', 'low']]
hl_df['kindex'] = np.arange(len(hl_df))

## 寻找k线高点

In [325]:
HH = pd.DataFrame()
HH['-2'] = hl_df.high.shift(2)
HH['-1'] = hl_df.high.shift(1)
HH['0'] = hl_df.high
HH['1'] = hl_df.high.shift(-1)
HH['2'] = hl_df.high.shift(-2)
HH = hl_df.loc[:, ['high', 'kindex']].loc[HH.idxmax(axis=1) == '0']
HH.columns = ['peak', 'kindex']

## 寻找k线低点

In [326]:
LL = pd.DataFrame()
LL['-2'] = hl_df.low.shift(2)
LL['-1'] = hl_df.low.shift(1)
LL['0'] = hl_df.low
LL['1'] = hl_df.low.shift(-1)
LL['2'] = hl_df.low.shift(-2)
LL = hl_df.loc[:, ['low', 'kindex']].loc[LL.idxmin(axis=1) == '0']
LL.columns = ['peak', 'kindex']
LL.peak = -LL.peak

## 获取高低分形DF

In [337]:
gdfx = pd.concat([HH, LL])
gdfx = gdfx.sort_index()

## 去除连续高点或者低点
高点不能比低点低，低点不能比高点高

In [338]:
def removed_continue_peak(gdfx):
    flag = pd.Series([False] * 3)
    while not flag.all():
        yflag = abs((gdfx.peak / gdfx.peak.shift(1)) ** (gdfx.peak / abs(gdfx.peak))) > 1
        tflag = abs((gdfx.peak / gdfx.peak.shift(-1)) ** (gdfx.peak / abs(gdfx.peak)) ) > 1
        flag = np.logical_and(yflag ,tflag)
        flag.iloc[[0,-1]] = True
        gdfx = gdfx[flag]
    return gdfx
gdfx = removed_continue_peak(gdfx)
gdfx.iloc[[0,-1]]

,peak,kindex
datetime,,
2013-10-18,984.0,0
2017-10-23,471.5,980


In [348]:
flag.iloc[[0,-1]] = True
flag.iloc[[0,-1]]
gdfx.iloc[[0,-1]]

datetime
2013-08-19    True
2017-10-16    True
Name: peak, dtype: bool

,peak,kindex
datetime,,
2013-10-18,984.0,0
2017-10-23,471.5,980


## 去除相邻较近的高低点

In [340]:
def removed_near_peak(gdfx, num):
    yflag = abs((gdfx.peak / gdfx.peak.shift(2)) ** (gdfx.peak / abs(gdfx.peak))) > 1
    tflag = abs((gdfx.peak / gdfx.peak.shift(-2)) ** (gdfx.peak / abs(gdfx.peak)) ) > 1
    nflag = gdfx.kindex - gdfx.kindex.shift(1) > num
    flag = np.logical_or(np.logical_and(yflag, tflag), nflag)
    flag.iloc[[0,-1]] = True
    return gdfx[flag]
gdfx = removed_near_peak(gdfx, 0)

In [341]:
gdfx = removed_continue_peak(gdfx)

In [352]:
instrument = Instrument()
code = 'ml8'
df = instrument.bar(code, asset='future')
hl_df = df.loc[:, ['high', 'low']]
hl_df['kindex'] = np.arange(len(hl_df))
ks_czsz = KsCzsz(hl_df)
ks_czsz.calculate_gdfx()
gdfx = ks_czsz.gdfx

AttributeError: 'KsCzsz' object has no attribute 'removed_continue_peak'

In [342]:
df_draw = df.reset_index()
ohlc = df_draw.loc[:, ['datetime', 'open', 'high', 'low', 'close']]
ohlc = ohlc.values.tolist()
volume = df_draw.loc[:, ['datetime','volume']]
volume = volume.values.tolist()
openInt = df_draw.loc[:, ['datetime','openInt']]
openInt = openInt.values.tolist()
gdfx_draw = abs(gdfx).reset_index()
peak = gdfx_draw.loc[:, ['datetime','peak']]
peak = peak.values.tolist()

In [343]:
chart = Highstock(width=1100, height=600)
chart.JSsource = [
    'Api/static/jquery/jquery-3.2.1.min.js',
    'Api/static/highcharts/highstock.js',
    'Api/static/highcharts/modules/exporting.js',
    'Api/static/highcharts/css/highcharts-more.js'
]
options = {
        'rangeSelector': {
            'selected': 1,
            'inputDateFormat': '%Y-%m-%d'
        },
        'title': {
            'text': code
        },
        'xAxis': {
            'dateTimeLabelFormats': {
                'millisecond': '%H:%M:%S.%L',
                'second': '%H:%M:%S',
                'minute': '%H:%M',
                'hour': '%H:%M',
                'day': '%m-%d',
                'week': '%m-%d',
                'month': '%y-%m',
                'year': '%Y'
            }
        },
        'yAxis': [{
            'labels': {
                'align': 'right',
                'x': -3
            },
            'title': {
                'text': '股价'
            },
            'height': '60%',
            'lineWidth': 2
        }, {
            'labels': {
                'align': 'right',
                'x': -3
            },
            'title': {
                'text': '成交量'
            },
            'top': '65%',
            'height': '35%',
            'offset': 0,
            'lineWidth': 2
        }],
}

chart.set_dict_options(options)
data_options = {
    'color': 'green',
    'lineColor': 'green',
    'upColor': 'red',
    'upLineColor': 'red',
    'tooltip': {},
}

In [344]:
chart.add_data_set(ohlc, series_type='candlestick', name=code, color='green', lineColor='green', upColor='red', upLineColor='red')
chart.add_data_set(volume, series_type='column', name='volume', yAxis=1)
chart.add_data_set(openInt , name='openInt', yAxis=1)

In [345]:
chart

In [346]:
gdfx_draw = abs(gdfx).reset_index()
peak = gdfx_draw.loc[:, ['datetime','peak']]
peak = peak.values.tolist()
chart.add_data_set(peak , name='peak')

In [347]:
chart